In [2]:
import numpy as np
import pandas as pd
import scipy as py


ModuleNotFoundError: No module named 'dimepy'

In [4]:
import sys
import base64
import zlib
import re
import numpy as np
import pandas as pd
from lxml import etree

def convert_to_safe_filename(string):
    #replace all not alphanum chars with underscore
    return re.sub(r'[^a-zA-Z0-9]', '_', string)

def run(file):
    #parse mzML and read namespace
    mzml = etree.parse(file)
    namespace = mzml.getroot().nsmap
    
    #iterate through pressure chromatograms
    for chrom in mzml.findall('//chromatogram/cvParam[@accession="MS:1003019"]/..', namespaces=namespace):
        chrom_name = chrom.get('id')
        decoded_arrays = dict()
        #iterate through binary arrays
        for array in chrom.findall('binaryDataArrayList/binaryDataArray', namespaces=namespace):
            #decode binary data
            binary = base64.b64decode(array.find('binary', namespaces=namespace).text.encode('ascii'))
            
            #parsing array parameters (quick and dirty approach)
            #set all to None
            kind = None
            compression = None
            size = None
            
            for param in array.findall('cvParam', namespaces=namespace):
                name = param.get('name').split()
                if name[-1] == 'array':
                    kind = name[0]
                elif name[-1] == 'compression':
                    compression = name[0]
                elif name[-1] == 'float':
                    size = name[0]
            
            #try parsing binary format
            if size == '64-bit':
                datatype='float64'
            elif size == '32-bit':
                datatype='float32'
            else:
                raise ValueError(f'Unknown binary format {size}')
            
            #try parsing compression and decode the array
            if compression == 'zlib':
                decoded_array = np.frombuffer(zlib.decompress(binary), dtype=datatype)
            elif compression == 'no':
                decoded_array = np.frombuffer(binary, dtype=datatype)
            else:
                raise ValueError(f'Cannot parse compression from {compression}')
            
            if not kind is None:
                decoded_arrays[kind] = decoded_array
            else:
                raise ValueError(f'Kind of array is not set')
                
        csv_path = file.replace('.mzML', f'_{convert_to_safe_filename(chrom_name)}.csv')
        pd.DataFrame(decoded_arrays).to_csv(csv_path, index=False)

if __name__ == '__main__':
    for file in sys.argv[1:]:
        try:
            run(file)
        except Exception as e:
            print (f'Processing of {file} failed; raised {e}')

Processing of --ip=127.0.0.1 failed; raised Error reading file '--ip=127.0.0.1': failed to load external entity "--ip=127.0.0.1"
Processing of --stdin=9008 failed; raised Error reading file '--stdin=9008': failed to load external entity "--stdin=9008"
Processing of --control=9006 failed; raised Error reading file '--control=9006': failed to load external entity "--control=9006"
Processing of --hb=9005 failed; raised Error reading file '--hb=9005': failed to load external entity "--hb=9005"
Processing of --Session.signature_scheme="hmac-sha256" failed; raised Error reading file '--Session.signature_scheme="hmac-sha256"': failed to load external entity "--Session.signature_scheme="hmac-sha256""
Processing of --Session.key=b"0d9b3aa4-868b-475c-b14e-a0d08c942bb8" failed; raised Error reading file '--Session.key=b"0d9b3aa4-868b-475c-b14e-a0d08c942bb8"': failed to load external entity "--Session.key=b"0d9b3aa4-868b-475c-b14e-a0d08c942bb8""
Processing of --shell=9007 failed; raised Error read

In [16]:
#!/home/tiago/anaconda3_old/bin/python
import pymzml
import re
import os
#folder of mzml
lst=os.listdir('/home/sanjay/github/lipids/Lipidomics/OzID')

for file in lst:
    if file.endswith('.mzML'):
        out = file.split(".")[0]+'.csv'
        #print (out)

        out_file=open("mzml"+out, 'w+')
        
        f = file
        print (f)

        msrun = pymzml.run.Reader(f)
        print(msrun)



        for spectrum in msrun:
            # exit()
            # continue
            m = re.search("Q1=([\d\.]+) Q3=([\d\.]+)", spectrum['id'])
            if m:
                out_file.write ("%s -> %s\n" %(m.group(1), m.group(2)) )
                out_file.write (",X(Minutes),Y(Counts)\n")
                cont = 0
                for mz, i in spectrum.peaks:
                    out_file.write ("%i,%s,%s\n" % (cont,mz, i))
                    cont = cont+1

mzml.mzML


In [3]:
#!/home/tiago/anaconda3_old/bin/python
import pymzml
import re
import os
#folder of mzml
lst=os.listdir('/home/sanjay/github/lipids/Lipidomics/OzID')

for file in lst:
    if file.endswith('.mzML'):
        out = file.split(".")[0]+'.csv'
        #print (out)

        out_file=open("output/"+out, 'w+')
        
        f = 'output/'+file
        print (f)

        msrun = pymzml.run.Reader(f)
        print(msrun)



        for spectrum in msrun:
            # exit()
            # continue
            m = re.search("Q1=([\d\.]+) Q3=([\d\.]+)", spectrum['id'])
            if m:
                out_file.write ("%s -> %s\n" %(m.group(1), m.group(2)) )
                out_file.write (",X(Minutes),Y(Counts)\n")
                cont = 0
                for mz, i in spectrum.peaks:
                    out_file.write ("%i,%s,%s\n" % (cont,mz, i))
                    cont = cont+1






output/mzml.mzML


OSError: Could not find obo file /home/sanjay/anaconda3/envs/lipid/lib/python3.7/site-packages/pymzml/obo/psi-ms-4.1.41.obo

In [11]:
pwd

'/home/sanjay/github/lipids/Lipidomics/OzID'